In [94]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import textwrap
import requests  # For making API calls
import os
from dotenv import load_dotenv
import openai

In [95]:
# --- Load API Key (from .env file or environment variables)---
load_dotenv()  # take environment variables from .env.
api_key = os.getenv("OPENAI_API_KEY")  # Recommend using a .env file

if api_key is None:
    raise ValueError("API key for OPEN AI is not set.  "
                     "Set it in a .env file or as an environment variable.")


In [96]:
# --- 1. Simulated User Input (Data Loading) ---
def simulate_satellite_data(num_points=100):
    #Same function from previous answer)
    np.random.seed(42)
    data = {
        'latitude': np.random.uniform(30, 50, num_points),
        'longitude': np.random.uniform(-125, -70, num_points),
        'ndvi': np.random.rand(num_points),
    }
    data['ndvi'] += 0.2 * np.sin(data['latitude'] * np.pi / 180)
    data['ndvi'] = np.clip(data['ndvi'], 0, 1)
    return pd.DataFrame(data)


In [97]:
# --- MODIFIED: Function to call the OpenAI API (Corrected for v1.0.0+) ---
def call_openai(prompt, model="gpt-3.5-turbo"):
    """Calls the OpenAI API (Chat Completions - Corrected for v1.0.0+)."""
    try:
        client = openai.OpenAI() # Use the client for openai>=1.0.0
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=350,
            n=1,
            stop=None,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()

    except openai.OpenAIError as e: #Corrected exception class
        print(f"OpenAI API error: {e}")
        return None

In [98]:
# --- 2. LLM Agent (Task Understanding and Code Generation - Simulated) ---
# Simulate LLM agent data retrieval
df = simulate_satellite_data()
print("-" * 40)
print("  Geospatial Analysis System (Simulated)")
print("-" * 40)
print("\nUser Query: What is the size of the vegetation area?")
print("\nSimulated Input Data (First 5 Rows):")
print(df.head())

model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(df[['latitude', 'longitude']], df['ndvi'])

results = permutation_importance(
    model, df[['latitude', 'longitude']], df['ndvi'],
    n_repeats=10, random_state=42
)
importance = results.importances_mean

----------------------------------------
  Geospatial Analysis System (Simulated)
----------------------------------------

User Query: What is the size of the vegetation area?

Simulated Input Data (First 5 Rows):
    latitude   longitude      ndvi
0  37.490802 -123.271395  0.763758
1  49.014286  -89.997427  0.235115
2  44.639879 -107.710421  0.302158
3  41.973170  -97.028612  1.000000
4  33.120373  -75.083844  0.715709


In [99]:
# --- Prompt (Slightly adjusted) ---
prompt = f"""
You are a geospatial analysis assistant. A user has asked:
"What is the size of the vegetation area?"

We've used a SIMPLIFIED model with latitude, longitude, and simulated NDVI
to estimate vegetation. A real analysis would use satellite imagery.  We
calculated feature importances:

Latitude Importance: {importance[0]:.4f}
Longitude Importance: {importance[1]:.4f}

Explain these results in simple terms. What does feature importance mean in
this context?  Based on these importances, is location strongly or weakly
related to vegetation? Acknowledge the simplification. DO NOT invent a
specific area size. Explain *why* latitude/longitude MIGHT influence
vegetation (climate, sunlight, etc.).  Be concise and factual.
"""


In [100]:

# --- Call the OpenAI API ---
explanation_raw = call_openai(prompt, model="gpt-4o-mini")  # Or "gpt-4"

if explanation_raw:
    wrapper = textwrap.TextWrapper(width=80)
    explanation_wrapped = wrapper.fill(text=explanation_raw)

    print("\n--- LLM Agent Response ---")
    print("\nPrompt:\n")
    print(wrapper.fill(text=prompt))

    print("\nExplanation:\n")
    print(explanation_wrapped)
else:
    print("\n--- LLM Agent Response ---")
    print("\nError: Could not retrieve explanation from OpenAI API.")

print("\n--- xAI Geospatial Results ---")
print("\nFeature Importance:")
print(f"  Latitude:  {importance[0]:.4f}")
print(f"  Longitude: {importance[1]:.4f}")
print("\n--- End of Simulation ---")


--- LLM Agent Response ---

Prompt:

 You are a geospatial analysis assistant. A user has asked: "What is the size of
the vegetation area?"  We've used a SIMPLIFIED model with latitude, longitude,
and simulated NDVI to estimate vegetation. A real analysis would use satellite
imagery.  We calculated feature importances:  Latitude Importance: 0.7208
Longitude Importance: 0.7737  Explain these results in simple terms. What does
feature importance mean in this context?  Based on these importances, is
location strongly or weakly related to vegetation? Acknowledge the
simplification. DO NOT invent a specific area size. Explain *why*
latitude/longitude MIGHT influence vegetation (climate, sunlight, etc.).  Be
concise and factual.

Explanation:

Feature importance in this context refers to how much each factor (latitude and
longitude) contributes to predicting the size of the vegetation area, as
indicated by the simulated NDVI (Normalized Difference Vegetation Index). Higher
importance values